In [22]:
'''dcca attack based on Minemura 2016
and scrambling dif DCs within their original size'''

import cv2
import random
import numpy as np

In [23]:
random.seed(0)

In [26]:
# def dcca_attack(image_name):
image_name = 'lena'
img_file = image_name + '.tif'
intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
height = np.shape(intensities)[0]
width = np.shape(intensities)[1]
imf = np.float32(intensities)
fsketch_intensities = np.zeros([height,width])

prev_dc_real = 0
for i in range(0,int(height/8)):
    for j in range(0,int(width/8)):
        blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
        shifted_blk = blk - 128 #level_shift for dct
        dct = cv2.dct(shifted_blk)
        nq_coef = np.round(dct)
        nq_coef_ac = np.delete(nq_coef, [0,0])
        nq_coef_dc = nq_coef[0,0]

        dc_real = nq_coef_dc
        dif_real = dc_real - prev_dc_real
        if dif_real == 0:
            enc_dif = 0
            fsketch = 0
        else: 
            dif_real_size = int(np.floor(np.log2(abs(dif_real)))+1)
            neg_range = range(-2**dif_real_size + 1, -2**(dif_real_size-1)+1)
            pos_range = range(2**(dif_real_size-1), 2**(dif_real_size))
            possible_difs_in_size = np.append(neg_range, pos_range)
            enc_dif = random.choice(possible_difs_in_size)
            
            fsketch_temp = np.floor(np.log2(abs(enc_dif)))
            fsketch = 2 ** fsketch_temp * 3
            
        prev_dc_real = nq_coef_dc           

        fsketch_blk = fsketch * np.ones([8, 8])
        fsketch_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= fsketch_blk

fsketch_modified = fsketch_intensities.copy()
fsketch_modified[fsketch_modified>255] = 255

filename = 'dcca_attack_category'+image_name+'.png'
cv2.imwrite(filename, fsketch_modified)
cv2.waitKey(1000)

#     return fsketch_modified

-1

In [27]:
512/8

64.0